## Import stuff

In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time
import sklearn.metrics as metrics


## Load training data from csv


In [6]:
val_data = pd.read_csv("val_distances.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)


val_data.drop("anchor_left_distance_mn", axis=1, inplace=True)
val_data.drop("anchor_right_distance_mn", axis=1, inplace=True)
for extractor in ["xl","resnet","clip","vit","merged","vit2","vit3","clip2"]:
    val_data[extractor] = val_data["anchor_left_distance_"+extractor]-val_data["anchor_right_distance_"+extractor]
    val_data.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
    val_data.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


X_val = np.array(val_data.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_val.shape)
print("Y shape: ", Y_val.shape)

val_data

X shape:  (7476, 8)
Y shape:  (7476,)


,xl,resnet,clip,vit,merged,vit2,vit3,clip2
0,-0.376129,-0.326017,-0.570965,-0.435678,-0.324292,-0.437379,-0.405525,-85.09982
1,-0.122618,-0.079811,-0.140376,-0.157509,-0.078924,-0.030681,0.067282,8.52122
2,0.261849,0.122114,-0.125301,0.011873,0.118544,-0.006831,0.411156,-57.96423
3,-0.547212,-0.657696,-1.082755,-1.271609,-0.993155,-1.251610,-1.061877,-141.26286
4,0.644307,0.251229,0.540372,-0.003367,0.064256,-0.044709,0.187157,30.28979
...,...,...,...,...,...,...,...,...
7471,0.105163,-0.299347,-0.152062,-0.087760,-0.040323,-0.124638,0.085406,-12.63158
7472,0.105163,-0.299347,-0.152062,-0.087760,-0.040323,-0.124638,0.085406,-12.63158
7473,0.105163,-0.299347,-0.152062,-0.087760,-0.040323,-0.124638,0.085406,-12.63158
7474,0.105163,-0.299347,-0.152062,-0.087760,-0.040323,-0.124638,0.085406,-12.63158


In [35]:
pred = []
for row in X_val:
    # score_left = 0.5*row[0]+row[1]+row[2]+0.5*row[3]+0.05*row[4]
    score_left = 0.5*row[0]+row[1]+row[2]+0.5*row[3]+0.05*row[4]
    pred.append(0 if score_left>0 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7839753879079722

In [22]:
pred = []
for row in X_val:
    score_left = 0
    score_left += row[0]>0
    score_left += row[1]>0
    score_left += row[2]>0
    score_left += row[3]>0
    score_left += row[4]>0
    pred.append(0 if score_left>2 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7748796147672552